# Session 1.1 (Part 2): Object Headers, Memory Model, and Optimization


In this notebook, we will go beyond object identity and dive into how Python objects are represented in memory. We'll explore:
- Reference counting and garbage collection
- Weak references
- Small integer caching and object reuse
- Memory layout of objects and how to inspect them
- The use of `__slots__` for memory optimization
- Best practices to avoid over-allocation

Each section includes step-by-step explanations, code, and commentary.


## 🔁 Reference Counting and Garbage Collection


Python uses reference counting to keep track of how many references point to an object. When the count drops to zero, the object is automatically deallocated. Python also has a cyclic garbage collector to deal with reference cycles.


In [ ]:

import sys
import gc

a = []
b = a

print("Reference count for a:", sys.getrefcount(a))  # Usually +1 because of getrefcount's own argument

# Create a circular reference
a.append(b)

# Delete both
del a
del b

# Force garbage collection
gc.collect()
print("Garbage collector stats:", gc.get_stats()[0])


Reference count for a: 3
Garbage collector stats: {'collections': 280, 'collected': 2305, 'uncollectable': 0}


## 👻 Weak References


Sometimes you want to reference an object without preventing it from being garbage collected. This is where `weakref` comes in.


In [ ]:

import weakref

class Demo:
    pass

obj = Demo()
r = weakref.ref(obj)

print("Original object:", obj)
print("Weak reference:", r)
print("Dereferenced weakref:", r())

# Delete strong reference
del obj
print("After deletion, weakref now returns:", r())


## 🔢 Small Integer Caching and Interning


Python caches small integers in the range [-5, 256]. These are reused to save memory and speed up execution.


In [ ]:

a = 100
b = 100
print("a is b (100):", a is b)

x = 1000
y = 1000
print("x is y (1000):", x is y)  # Likely False, not cached

print("id(a):", id(a), "id(b):", id(b))
print("id(x):", id(x), "id(y):", id(y))


## 📦 Memory Overhead: `sys.getsizeof` and `__slots__`


Every Python object has overhead. You can inspect object size using `sys.getsizeof`.

By default, classes allow dynamic attribute creation via `__dict__`. This adds memory overhead. You can reduce this by defining `__slots__`, which pre-declares fixed attributes and avoids `__dict__`.


In [ ]:

import sys

class Normal:
    def __init__(self):
        self.a = 1
        self.b = 2

class Slotted:
    __slots__ = ('a', 'b')
    def __init__(self):
        self.a = 1
        self.b = 2

n = Normal()
s = Slotted()

print("Size of normal class:", sys.getsizeof(n))
print("Size of slotted class:", sys.getsizeof(s))
print("Normal class dict:", n.__dict__)
# The following would raise an AttributeError if uncommented
# print(s.__dict__)


## ✅ Best Practices


1. Use `sys.getrefcount()` to debug reference cycles or leaks.
2. Prefer `weakref` for caches or registries where objects can be collected.
3. Understand Python's interning/caching to avoid identity traps.
4. Use `__slots__` for classes with many instances to reduce memory.
5. Avoid creating unnecessary references in tight loops or recursive calls.

These practices help reduce memory overhead and improve scalability for large-scale applications.


### 📌 Summary


- Python objects have a header containing metadata like reference count and type.
- Garbage collection helps clear cycles; `gc.collect()` forces it.
- `weakref` allows referencing without extending lifetime.
- Python reuses small ints and short strings for efficiency.
- `__slots__` is a powerful way to reduce per-object memory footprint.

By understanding these internals, you can write faster, more memory-efficient Python code.


In [1]:
#lets recap some stuff before we go hard core
# Integer object
x = 10
print(type(x))  # Output: <class 'int'>

# String object
y = "Hello"
print(type(y))  # Output: <class 'str'>

# List object
z = [1, 2, 3]
print(type(z))  # Output: <class 'list'>

# Function object
def my_function():
    pass

print(type(my_function))  # Output: <class 'function'>

# Class object
class MyClass:
    pass

obj = MyClass()
print(type(obj))  # Output: <class '__main__.MyClass'>
print(type(None))  # Output: <class 'NoneType'>
import math
print(type(math))  # Output: <class 'module'>


<class 'int'>
<class 'str'>
<class 'list'>
<class 'function'>
<class '__main__.MyClass'>
<class 'NoneType'>
<class 'module'>


In [2]:
# we can force seperate objects
a = int("257")
b = int("257")

print(a == b)  # True (values are equal)
print(a is b)  # False (different objects in memory)

a = "hello"
b = " ".join(["hello"])  # Dynamically created string

print(a == b)  # True (values are equal)
print(a is b)  # False (different objects in memory)

a = 3.14
b = 3.14

print(a == b)  # True (values are equal)
print(a is b)  # False (different objects in memory)

True
False
True
True
True
False
